In [1]:
import cobra
import re
import math
import json
import pandas as pd
from cobra import Model, Reaction, Metabolite
from cobra.io.dict import model_to_dict
from script.quality_control_function import *

In [2]:
# initial universal model from BiGG database
model = cobra.io.load_json_model('data/universal_model.json')

In [3]:
model

Name,bigg_universal
Memory address,0x07f7dadbc59e8
Number of metabolites,15638
Number of reactions,28301
Number of groups,0
Objective expression,0
Compartments,


# 1. Model preprocessing

## 1.1 Adding charge and formula for metabolites

In [4]:
met_108_df = pd.read_excel('data/met.xlsx',index_col='id')

model_dic = model_to_dict(model, sort=False)  
for met in model.metabolites:
    for met_dic in model_dic['metabolites']:  
        if met_dic['id'] == met.id:
            met_dic['charge'] = met_108_df.loc[met.id]['charge']
            met_dic['formula'] = met_108_df.loc[met.id]['formula']  
model = cobra.io.dict.model_from_dict(model_dic)  

## 1.2 Determining the direction of reaction

In [5]:
# exchange reactions
for rxn in model.reactions:
    if not exchange_rxn(model,rxn.id):
        model.reactions.get_by_id(rxn.id).bounds = (0,1000)
model.reactions.get_by_id('EX_glc__D_e').bounds = (-10,1000) # glucose
medium_rxn = ['EX_co2_e','EX_o2_e','EX_h_e','EX_h2o_e','EX_nh4_e','EX_pi_e','EX_so4_e','EX_fe3_e', 'EX_mn2_e','EX_fe2_e','EX_zn2_e', 'EX_mg2_e', 'EX_ca2_e', 'EX_ni2_e', 'EX_cu2_e', 'EX_cobalt2_e', 'EX_sel_e','EX_mobd_e', 'EX_k_e', 'EX_na1_e','EX_cl_e', 'EX_tungs_e', 'EX_slnt_e']
for ri in medium_rxn:
    rxn = model.reactions.get_by_id(ri)
    rxn.bounds = (-1000,1000)

###  1.2.1  directions in 108 GEMs

In [6]:
# Reaction directions were initially determined based on the maximum count among the three directions (forward, backward, and reversible) in the GEMs

rxn_109_bounds_df=pd.read_excel('data/bigg_108_models_rxn_bounds.xlsx',index_col='id')

# Except for exchange reactions, each reaction direction is determined based on the number of counts in 108 GEMs
no_direction_rxn=[] 
for r in model.reactions:
    direction_number=[]
    if not exchange_rxn(model,r.id):
        reversible=rxn_109_bounds_df.loc[r.id]['reversible']
        forword=rxn_109_bounds_df.loc[r.id]['forword direction']
        backword=rxn_109_bounds_df.loc[r.id]['reverse direction']
        direction_number.append(reversible)
        direction_number.append(forword)
        direction_number.append(backword)
        
        max_index=direction_number.index(max(direction_number))
        
        if max_index==1 and direction_number[1]==direction_number[2]:
            model.reactions.get_by_id(r.id).bounds=(-1000,1000)
        
        if max_index==0: # reversible
            model.reactions.get_by_id(r.id).bounds=(-1000,1000)
        
        if max_index==1:# forward
            model.reactions.get_by_id(r.id).bounds=(0,1000)
            
        if max_index==2:# backward
            model.reactions.get_by_id(r.id).bounds=(-1000,0)   
            
        if reversible==0 and forword==0 and backword==0:
            model.reactions.get_by_id(r.id).bounds=(-1000,1000)
            no_direction_rxn.append(r.id)

### 1.2.2 thermodynamic 

In [7]:
# the frequency of reaction directions
rxn_108_except_exchange_df=pd.read_excel('data/bigg_108_models_rxn_bounds_except_exchange_rxn.xlsx',index_col='id')
unrealiable_rxn = []
for ri in rxn_108_except_exchange_df.index:
    if rxn_108_except_exchange_df.loc[ri,'sum'] <= 10:
        unrealiable_rxn.append(ri)
    if rxn_108_except_exchange_df.loc[ri,'sum'] > 10 and rxn_108_except_exchange_df.loc[ri,'max_direction_frequency'] < 0.7:
        unrealiable_rxn.append(ri)  

# detaG 
detaG_correct = []
detaG_df = pd.read_excel('data/bigg_equilibrator3.0_select.xlsx',index_col = 'id')
for ri in detaG_df.index:
    if ri.startswith('icw-'):
        detaG_df.drop(ri,inplace=True)
for ri in detaG_df.index:
    if ri in unrealiable_rxn and not transport_rxn(model,ri):
        rxn = model.reactions.get_by_id(ri)
        g1 = str(detaG_df.loc[ri,'G1']).split(') ')[0].split('(')[1].split(' +')[0]
        g2 = str(detaG_df.loc[ri,'G2']).split(') ')[0].split('(')[1].split(' +')[0]
        if float(g1) < 0 and float(g2) < 0:
            if rxn.bounds!=(0,1000):
                rxn.bounds=(0,1000)
                detaG_correct.append(ri)
                
        if float(g1) > 0 and float(g2) > 0:
            if rxn.bounds!=(-1000,0):
                rxn.bounds=(-1000,0)
                detaG_correct.append(ri)

### 1.2.3 heuristic rules

In [8]:
# for reactions containing oxygen, their directions were assumed to be that of the oxygen consumption
O2_rxn = o2_rule(model,detaG_correct)

# most reactions cannot proceed in the direction of CO2 fixation, except for naturally known carbon fixation reactions and reactions utilizing CO2 and high-energy substrates such as phosphoenolpyruvate (PEP) and ATP
co2_rule_rxn = co2_rule(model,detaG_correct)

# atp 规则
ATP_pi = atp_rule(model,detaG_correct)

# most reactions produced  NH3/NH4+, except when it reacted with ATP, 2-oxoglutarate, chorismate, 5-phospho-alpha-D-ribose-1-diphosphate, and UDP-N-acetyl-beta-L-fucosamine
nh4_rule_rxn = nh4_rule(model,detaG_correct,O2_rxn)

# the direction of reactions with different compartment is same

rxnc_direction = modify_direction_compartment(model)
# print(len(rxnc_direction)) 

In [10]:
# check_model
model_dic = model_to_dict(model, sort=False)
with open('results/check_model.json', 'w') as f:
    json.dump(model_dic,f)

# 2. Error elimination

## 2.1  infinite generation of metabolites 

In [11]:
# penalizing reactions
rxn_grate_df = pd.read_excel('data/rxn_grate.xlsx',index_col='id')
rxn_grate={}
for rxn in model.reactions:
    if rxn.id in list(rxn_grate_df.index):
        rxn_grate[rxn.id]=rxn_grate_df.loc[rxn.id,'grate']

# unbalance reaction
balance_rxn,chargenan_rxn,H_balance_rxn,H2O_balance_rxn,unbalance_rxn = check_rxn_balance_number(model)

model.reactions.get_by_id('EX_glc__D_e').bounds = (0,1000)

In [12]:
check_model = cobra.io.load_json_model('results/check_model.json')
outputfile = 'results/quality_control/met/'

In [13]:
# metabolites containing carbon
element_id = 'co2'
c_met = check_carbon_production(model,element_id,rxn_grate,check_model,outputfile,unbalance_rxn)


4crsol_c 1000.0
1
BIOMASS_SA_nuc_only: 3.11 amp_c + 2.2 cmp_c + 0.76 damp_c + 0.53 dcmp_c + 0.6 dgmp_c + 0.73 dtmp_c + 4.01 gmp_c + 2.39 ump_c --> h2o_c 5
SULRSI: 3.0 h2o_c + 3.0 nadp_c + slnt_c --> 4.0 h_c + 3.0 nadph_c + sel_c 5
BIOMASS_SA_only_AA: 6.6 ala__L_c + 3.88 arg__L_c + 4.48 asn__L_c + 4.48 asp__L_c + 0.79 cys__L_c + 8.72 gln__L_c + 8.72 glu__L_c + 3.51 gly_c + 1.45 his__L_c + 4.06 ile__L_c + 5.21 leu__L_c + 5.45 lys__L_c + 1.94 met__L_c + 3.33 phe__L_c + 2.12 pro__L_c + 2.6 ser__L_c + 2.54 thr__L_c + 1.27 trp__L_c + 2.3 tyr__L_c + 4.12 val__L_c --> h2o_c 5
P5CDEG: e4hglu_c + nad_c --> 4hglusa_c + nadh_c 5
2
SFRR: h_c + so3_c --> h2o_c + h2s_c 5
FPRA: fdxrd_c + h_c + nadp_c --> fdxox_c + nadph_c 5
SIRA2: 6.0 fdxox_c + 3.0 h2o_c + h2s_c --> 6.0 fdxrd_c + 8.0 h_c + so3_c 5
GLGE: malt1p_c --> 2.0 14glucan_c + pi_c 5
ACCOAS: aacoa_c + coa_c --> accoa_c 5
3
CYTK2_2: atp_c + ctp_c + dcmp_c --> adp_c + cdp_c + dcdp_c 5
PPS2: air_c + amet_c --> 2mahmp_c + co_c + dad_2_c + for_c + me

In [16]:
# etabolites containing phosphorus
element_id = 'pi'
pi_met = check_other_elment_production(model,element_id,rxn_grate,check_model,outputfile,unbalance_rxn)

pi_c 1000.0
1


BIOMASS_SA_nuc_only: 3.11 amp_c + 2.2 cmp_c + 0.76 damp_c + 0.53 dcmp_c + 0.6 dgmp_c + 0.73 dtmp_c + 4.01 gmp_c + 2.39 ump_c --> h2o_c 5
CYTK2_2: atp_c + ctp_c + dcmp_c --> adp_c + cdp_c + dcdp_c 5
kaasIII: accoa_c + h_c + malacp_c --> aaacp_c + co2_c + coa_c 5
GLUSfx: akg_c + 2.0 fdxrd_c + gln__L_c + 2.0 h_c --> 2.0 fdxox_c + 2.0 glu__L_c 5
SULRSI: 3.0 h2o_c + 3.0 nadp_c + slnt_c --> 4.0 h_c + 3.0 nadph_c + sel_c 5
ALAB: akg_c + ala_B_c --> glu__L_c + octdp_c 5
DECACP: acACP_c + 4.0 malACP_c + 8.0 nadph_c --> 4.0 ACP_c + 4.0 co2_c + decacp_c + 8.0 nadp_c 5
OCTEACP: acACP_c + 8.0 malACP_c + 15.0 nadph_c --> 8.0 ACP_c + 8.0 co2_c + 15.0 nadp_c + octeacp_c 5
PEPHL: pe160_c --> 2agpe160_c + 0.041 bhdodec_c + 0.015 decacid_c + 0.048 dodecacid_c + 0.017 hepadecacid_c + 0.016 hepedecacid_c + 0.281 hexadecacid_c + 0.192 hexedecacid_c + 0.008 octadecacid_c + 0.375 octedecacid_c + 0.003 pendecacid_c + 0.003 tetdecacid_c 5
PLCG: pg160_c --> 12dgr160_c + glyc3p_c 5
CAV: 0.163 coa_c + 0.159 fad_c 

In [18]:
# metabolites containing nitrogen
element_id = 'nh4'
nh4_met = check_other_elment_production(model,element_id,rxn_grate,check_model,outputfile,unbalance_rxn)


nh4_c 1000.0
1


HPOXR: 2.0 h2o2_c --> 2.0 h2o_c + 2.0 o2_c 5
SULR_syn_1: fdxo_2_2_c + h2o_c + h2s_c --> fdxrd_c + h_c + so3_c 5
BIOMASS_SA_only_AA: 6.6 ala__L_c + 3.88 arg__L_c + 4.48 asn__L_c + 4.48 asp__L_c + 0.79 cys__L_c + 8.72 gln__L_c + 8.72 glu__L_c + 3.51 gly_c + 1.45 his__L_c + 4.06 ile__L_c + 5.21 leu__L_c + 5.45 lys__L_c + 1.94 met__L_c + 3.33 phe__L_c + 2.12 pro__L_c + 2.6 ser__L_c + 2.54 thr__L_c + 1.27 trp__L_c + 2.3 tyr__L_c + 4.12 val__L_c --> h2o_c 5
PROTEIN: 0.488 ala__L_c + 0.281 arg__L_c + 0.229 asn__L_c + 0.229 asp__L_c + 40.0 atp_c + 0.087 cys__L_c + 0.25 gln__L_c + 0.25 glu__L_c + 0.582 gly_c + 0.09 his__L_c + 0.276 ile__L_c + 0.428 leu__L_c + 0.326 lys__L_c + 0.146 met__L_c + 0.176 phe__L_c + 0.21 pro__L_c + 0.205 ser__L_c + 0.241 thr__L_c + 0.054 trp__L_c + 0.131 tyr__L_c + 0.402 val__L_c --> 40.0 adp_c + 40.0 pi_c + protein_c 5
2
NALN6_1: akg_c + nal2a6o_c --> glu__L_c + n6all26d_c 5
3
NP1_2: nicrns_c + pi_c --> h_c + ncam_c + r1p_c 5
4
THDPS_2: h2o_c + succoa_c + thdp_c --> 

In [20]:
# metabolites containing sulfur
element_id = 'so4'
so4_met = check_other_elment_production(model,element_id,rxn_grate,check_model,outputfile,unbalance_rxn)


so4_c 1000.0
1
epo_degradation: 26.0 h2o_c + peptide_c --> ala__L_c + cys__L_c + glu__L_c + 3.0 gly_c + his__L_c + 10.0 leu__L_c + met__L_c + 3.0 pro__L_c + 2.0 ser__L_c + 2.0 trp__L_c + 2.0 val__L_c 5
HPOXR: 2.0 h2o2_c --> 2.0 h2o_c + 2.0 o2_c 5
SULR_syn_1: fdxo_2_2_c + h2o_c + h2s_c --> fdxrd_c + h_c + so3_c 5
BIOMASS_SA_only_AA: 6.6 ala__L_c + 3.88 arg__L_c + 4.48 asn__L_c + 4.48 asp__L_c + 0.79 cys__L_c + 8.72 gln__L_c + 8.72 glu__L_c + 3.51 gly_c + 1.45 his__L_c + 4.06 ile__L_c + 5.21 leu__L_c + 5.45 lys__L_c + 1.94 met__L_c + 3.33 phe__L_c + 2.12 pro__L_c + 2.6 ser__L_c + 2.54 thr__L_c + 1.27 trp__L_c + 2.3 tyr__L_c + 4.12 val__L_c --> h2o_c 5
AHSERL3: acser_c + seln_c --> ac_c + scys__L_c 5
SELTOR_c: 6.0 fdxrd_c + 7.0 h_c + slnt_c --> 6.0 fdxox_c + 3.0 h2o_c + seln_c 5
FPRA: fdxrd_c + h_c + nadp_c --> fdxox_c + nadph_c 5
SIRA2: 6.0 fdxox_c + 3.0 h2o_c + h2s_c --> 6.0 fdxrd_c + 8.0 h_c + so3_c 5
P5CCD: glu__L_c + h_c + nadh_c --> glu5sa_c + nad_c + pi_c 5
PROTEIN: 0.488 ala__L_c 

SUCptspp_1: pep_c --> pyr_c + suc6p_c 5
r1383: HC02129_c --> 6.0 glu__L_c + thf_c 5
ATPB: atp_c --> adp_c + pi_c 5
GLYFEM: glyclt_c + h_c --> ac_c + co2_c + h2o_c + succ_c 5
r1382: 6.0 atp_c + 6.0 glu__L_c + 6.0 h2o_c + thf_c --> HC02129_c + 6.0 adp_c + 6.0 pi_c 5
NALN6_1: akg_c + nal2a6o_c --> glu__L_c + n6all26d_c 5
ASPA1: Lcyst_c + akg_c --> ac_c + glu__L_c 5
G3POACTF: 0.041 bhdodec_c + 0.015 decacp_c + 0.048 dodecacp_c + glyc3p_c + 0.017 hepacp_c + 0.016 hepeacp_c + 0.281 hexacp_c + 0.192 hexeacp_c + 0.008 octacp_c + 0.375 octeacp_c + 0.003 penacp_c + 0.003 tetacp_c --> 0.958 ACP_c + aglp_c 5
AGLPACTF: aglp_c + 0.041 bhdodec_c + 0.015 decacp_c + 0.048 dodecacp_c + 0.017 hepacp_c + 0.016 hepeacp_c + 0.281 hexacp_c + 0.192 hexeacp_c + 0.008 octacp_c + 0.375 octeacp_c + 0.003 penacp_c + 0.003 tetacp_c --> 0.958 ACP_c + 0.958 pa160_c 5
3
PHBS_syn_1: 3hbcoa__R_c --> coa_c + phb_c 5
OXPTNDH_1: nad_c + oxptn_c --> glutar_c + nadh_c 5
ACCOAS: aacoa_c + coa_c --> accoa_c 5
4
ARTCOAL1: Rtota

In [22]:
# metabolites containing oxygen
element_id = 'o2'
o2_met = check_other_elment_production(model,element_id,rxn_grate,check_model,outputfile,unbalance_rxn)
model.reactions.get_by_id('EX_o2_e').bounds = (-1000,1000)


o2_c 666.6666666666665
1


r1383: HC02129_c --> 6.0 glu__L_c + thf_c 5
SULRSI: 3.0 h2o_c + 3.0 nadp_c + slnt_c --> 4.0 h_c + 3.0 nadph_c + sel_c 5
HPOXR: 2.0 h2o2_c --> 2.0 h2o_c + 2.0 o2_c 5
GLYOX_2: h2o_c + lgt__S_c --> gthrd_c + h_c + lac__D_c + mthgxl_c 5
r1382: 6.0 atp_c + 6.0 glu__L_c + 6.0 h2o_c + thf_c --> HC02129_c + 6.0 adp_c + 6.0 pi_c 5
GLGE: malt1p_c --> 2.0 14glucan_c + pi_c 5
G3POACTF: 0.041 bhdodec_c + 0.015 decacp_c + 0.048 dodecacp_c + glyc3p_c + 0.017 hepacp_c + 0.016 hepeacp_c + 0.281 hexacp_c + 0.192 hexeacp_c + 0.008 octacp_c + 0.375 octeacp_c + 0.003 penacp_c + 0.003 tetacp_c --> 0.958 ACP_c + aglp_c 5
AGLPACTF: aglp_c + 0.041 bhdodec_c + 0.015 decacp_c + 0.048 dodecacp_c + 0.017 hepacp_c + 0.016 hepeacp_c + 0.281 hexacp_c + 0.192 hexeacp_c + 0.008 octacp_c + 0.375 octeacp_c + 0.003 penacp_c + 0.003 tetacp_c --> 0.958 ACP_c + 0.958 pa160_c 5
CPS_1: atp_c + h_c + hco3_c + nh4_c --> adp_c + cbp_c + pi_c 5
r1383 1
SULRSI 2
HPOXR 3
HPOXR: 2.0 h2o2_c --> 2.0 h2o_c + 2.0 o2_c unbalance {'O': 2.0

In [24]:
# all reactions for the infinite generation of metabolites
met_net_generation = list(set(c_met + pi_met + nh4_met + so4_met + o2_met))
cor_unri,cor_unrr,cor_balri,cor_balrr,cor_type,del_ri,del_rr,del_type = modify_delete_rxn(model,met_net_generation,balance_rxn)

64
NALN6_1: akg_c + nal2a6o_c --> glu__L_c + n6all26d_c ADAPAT: glu__L_c + nal2a6o_c <=> akg_c + n6all26d_c 1
ACCOAS: aacoa_c + coa_c <=> accoa_c ACACT1r: 2.0 accoa_c <=> aacoa_c + coa_c 1
FACT: 2obut_c + coa_c <=> for_c + 2.0 ppcoa_c OBTFL: 2obut_c + coa_c --> for_c + ppcoa_c 1
CPS_1: atp_c + h_c + hco3_c + nh4_c --> adp_c + cbp_c + pi_c CBPSam_1: 2.0 atp_c + hco3_c + nh4_c --> 2.0 adp_c + cbp_c + 2.0 h_c + pi_c 1
HPOXR: 2.0 h2o2_c --> 2.0 h2o_c + 2.0 o2_c CAT: 2.0 h2o2_c --> 2.0 h2o_c + o2_c 1
SLGTH: gthrd_c + h_c + slnt_c --> dgslnt_c + gthox_c + h2o_c SELGTHR: 4.0 gthrd_c + 2.0 h_c + slnt_c --> dgslnt_c + gthox_c + 3.0 h2o_c 1


## 2.2 infinite generation of reducing equivalents 

In [26]:
# NADH

add_rxn(model,'ADD_nadh_c','nadh_c --> nad_c + h_c')
rxn_grate['ADD_nadh_c'] = 0

objective_reaction = 'ADD_nadh_c'
model.objective='ADD_nadh_c'
nadh_flux = model.optimize().fluxes['ADD_nadh_c']

if nadh_flux > 1e-6:
    objective_reaction = 'ADD_nadh_c'
    constraint_value = 1e-6
    grate_delete_rxn = find_infinite_rxn(model,objective_reaction,constraint_value,rxn_grate)
    outputfile = 'results/quality_control/nadh/'
    nadh_end_rxn = locating_rxn(model,check_model,objective_reaction,constraint_value,grate_delete_rxn,outputfile,unbalance_rxn)
else:
    pass

rxn = model.reactions.get_by_id('ADD_nadh_c')
model.remove_reactions([rxn])

1
OXPTNDH_1: nad_c + oxptn_c --> glutar_c + nadh_c 5
FPRA: fdxrd_c + h_c + nadp_c --> fdxox_c + nadph_c 5
P5CDEG: e4hglu_c + nad_c --> 4hglusa_c + nadh_c 5
2


CY_focytb5_c: ficytb5_c --> focytb5_c 5
FDR: nadph_c + rdxo_c --> h_c + nadp_c + rdxr_c 5
3
SULR_syn_1: fdxo_2_2_c + h2o_c + h2s_c --> fdxrd_c + h_c + so3_c 5
SFRR: h_c + so3_c --> h2o_c + h2s_c 5
FNRR2: fdxrd_c + h_c + nadp_c --> fdxo_2_2_c + nadph_c 5
4
PTGFS: prostgh2_c + trdox_c --> prostgf2_c + trdrd_c 5
5
FRDO: fdxrd_c + nadp_c --> fdxox_c + h_c + nadph_c 4
HYDA: 2.0 fdxrd_c --> 2.0 fdxox_c + h2_c + 2.0 h_c 4
GULNLR: guln__L_c + 2.0 nad_c --> fruur_c + 2.0 nadh_c 4
6
RE3477C: h2o_r + txa2_r --> CE1447_c + 2.0 h_c 5
7
GLA: nad_c + udpgalur_c --> 2.0 h_c + nadh_c + udpglcur_c 5
8
SULRSI: 3.0 h2o_c + 3.0 nadp_c + slnt_c --> 4.0 h_c + 3.0 nadph_c + sel_c 5
ATPB: atp_c --> adp_c + pi_c 5
MECDPDH4E: 2mecdp_c + fdxrd_c + h_c --> fdxo_2_2_c + h2mb4p_c + h2o_c 5
SUCD2: fad_c + 2.0 h_c + mqn7_c + succ_c --> fadh2_c + fum_c + mql7_c 5
MRF: fdxrd_c + 3.0 h_c + mlthf_c --> 5mthf_c + fdxo_2_2_c 5
9
AHMMPS: air_c + 2.0 h_c --> 4ahmmp_c + gcald_c + pi_c 5
SULRR2: h_c + nadph_c + so3_c --> h2o_c 

In [28]:
# NADPH
add_rxn(model,'ADD_nadph_c','nadph_c --> nadp_c + h_c')
objective_reaction = 'ADD_nadph_c'
model.objective='ADD_nadph_c'
nadph_flux = model.optimize().fluxes['ADD_nadph_c']

if nadph_flux > 1e-6:
    objective_reaction = 'ADD_nadph_c'
    constraint_value = 1e-6
    grate_delete_rxn = find_infinite_rxn(model,objective_reaction,constraint_value,rxn_grate)
    nadph_end_rxn = locating_rxn(model,check_model,objective_reaction,constraint_value,grate_delete_rxn,outputfile,unbalance_rxn)
else:
    nadph_end_rxn = []

rxn = model.reactions.get_by_id('ADD_nadph_c')
model.remove_reactions([rxn])

In [29]:
# fadh2

add_rxn(model,'ADD_fadh2_c','fadh2_c --> fad_c + h_c')
objective_reaction = 'ADD_fadh2_c'
model.objective='ADD_fadh2_c'
fadh2_flux = model.optimize().fluxes['ADD_fadh2_c']

if fadh2_flux > 1e-6:
    objective_reaction = 'ADD_fadh2_c'
    constraint_value = 1e-6
    grate_delete_rxn = find_infinite_rxn(model,objective_reaction,constraint_value,rxn_grate)
    fadh2_end_rxn = locating_rxn(model,check_model,objective_reaction,constraint_value,grate_delete_rxn,outputfile,unbalance_rxn)
else:
    fadh2_end_rxn = []

rxn = model.reactions.get_by_id('ADD_fadh2_c')
model.remove_reactions([rxn])


In [30]:
# q8h2

add_rxn(model,'ADD_q8h2_c','q8h2_c --> q8_c + h_c')
objective_reaction = 'ADD_q8h2_c'
model.objective='ADD_q8h2_c'
q8h2_flux = model.optimize().fluxes['ADD_q8h2_c']

if q8h2_flux > 1e-6:
    objective_reaction = 'ADD_q8h2_c'
    constraint_value = 1e-6
    grate_delete_rxn = find_infinite_rxn(model,objective_reaction,constraint_value,rxn_grate)
    q8h2_end_rxn = locating_rxn(model,check_model,objective_reaction,constraint_value,grate_delete_rxn,outputfile,unbalance_rxn)
else:
    q8h2_end_rxn = []

rxn = model.reactions.get_by_id('ADD_q8h2_c')
model.remove_reactions([rxn])

In [31]:
# 2dmmql8

add_rxn(model,'ADD_2dmmql8_c','2dmmql8_c --> 2dmmq8_c + h_c')
objective_reaction = 'ADD_2dmmql8_c'
model.objective='ADD_2dmmql8_c'
dmmql8_flux = model.optimize().fluxes['ADD_2dmmql8_c']

if dmmql8_flux > 1e-6:
    objective_reaction = 'ADD_2dmmql8_c'
    constraint_value = 1e-6
    grate_delete_rxn = find_infinite_rxn(model,objective_reaction,constraint_value,rxn_grate)
    dmmql8_end_rxn = locating_rxn(model,check_model,objective_reaction,constraint_value,grate_delete_rxn,outputfile,unbalance_rxn)
else:
    dmmql8_end_rxn = []

rxn = model.reactions.get_by_id('ADD_2dmmql8_c')
model.remove_reactions([rxn])

In [32]:
met_rxn = nadh_end_rxn + nadph_end_rxn + fadh2_end_rxn + q8h2_end_rxn + dmmql8_end_rxn
cor_unri,cor_unrr,cor_balri,cor_balrr,cor_type,del_ri,del_rr,del_type = modify_delete_rxn(model,met_rxn,balance_rxn)

P5CDEG: e4hglu_c + nad_c --> 4hglusa_c + nadh_c 4HGLSD: 4hglusa_c + h2o_c + nad_c --> e4hglu_c + 2.0 h_c + nadh_c 2
FDR: nadph_c + rdxo_c --> h_c + nadp_c + rdxr_c RDXR: nadh_c + 2.0 rdxo_c <=> h_c + nad_c + 2.0 rdxr_c 2
SFRR: h_c + so3_c --> h2o_c + h2s_c SULR: 5.0 h_c + 3.0 nadph_c + so3_c --> 3.0 h2o_c + h2s_c + 3.0 nadp_c 2
GLA: nad_c + udpgalur_c --> 2.0 h_c + nadh_c + udpglcur_c UGE: udpglcur_c <=> udpgalur_c 2


## 2.3 infinite generation of energy 

In [33]:
# deleting proton gradient reactions
respiratory_rxn=[]
ATP_list=['atp','adp','h','pi','h2o','h']
for r in model.reactions:
    ATP_mets=[]
    for met in r.metabolites:
        if str(met)[-2:-1]=='_':
            ATP_mets.append(str(met)[:-2])
        elif str(met)[-3:-2]=='_':
            ATP_mets.append(str(met)[:-3])
    if sorted(ATP_mets)==sorted(ATP_list):
        respiratory_rxn.append(r.id)
        r.bounds = (0,0)

for rei in list(set(respiratory_rxn)):
    rxn = model.reactions.get_by_id(rei)
    model.remove_reactions([rxn]) 

In [34]:
# adding combined respiratory chain reactions

reaction = Reaction('ADD_respiratory1') 
model.add_reactions([reaction])
reaction.build_reaction_from_string('7.0 adp_c + 11.0 h_c + 4.0 nadh_c + 2.0 o2_c + 7.0 pi_c --> 7.0 atp_c + 11.0 h2o_c + 4.0 nad_c')

reaction = Reaction('ADD_respiratory2') 
model.add_reactions([reaction])
reaction.build_reaction_from_string('adp_c + fadh2_c + h_c + o2_c + pi_c --> atp_c + fad_c + h2o_c')

reaction = Reaction('ADD_respiratory3') 
model.add_reactions([reaction])
reaction.build_reaction_from_string('adp_c + h_c + 0.5 o2_c + pi_c + q8h2_c --> atp_c + 2.0 h2o_c + q8_c')

add_rxn(model,'ADD_h_c','h_c <=> ')

rxn_grate['ADD_respiratory1'] = 0
rxn_grate['ADD_respiratory2'] = 0
rxn_grate['ADD_respiratory3'] = 0
rxn_grate['ADD_h_c'] = 0

In [35]:
# atp
model.objective='ATPM'
atp_flux = model.optimize().fluxes['ATPM']

if atp_flux > 1e-6:
    objective_reaction = 'ATPM'
    constraint_value = 1e-6
    grate_delete_rxn = find_infinite_rxn(model,objective_reaction,constraint_value,rxn_grate)
    outputfile = 'results/quality_control/energy/'
    atp_rxn = locating_rxn(model,check_model,objective_reaction,constraint_value,grate_delete_rxn,outputfile,unbalance_rxn)

1


FE2Gabcpp: fe2_p + gtp_c + h2o_c --> fe2_c + gdp_c + h_c + pi_c 2
2
RE2649C: h2o_c + ppcoa_c --> coa_c + h_c + ppa_c 2
3
FE2GTPabc: fe2_e + gtp_c + h2o_c --> fe2_c + gdp_c + h_c + pi_c 2
4
PPPPH: gbdp_c + h_c + pi_c --> gdptp_c + h2o_c 2
PpGppP: gdp_c + ppi_c --> gbdp_c + h2o_c 2
5
RE3629C: CE2089_c + accoa_c --> CE2088_c + coa_c + h_c 2
RE3630C: CE2088_c + h2o_c --> CE2089_c + ac_c 2
6
PPAAFDOR: fdxo_42_c + h2o_c + ppal_c --> fdxr_42_c + 3.0 h_c + ppa_c 2
FRDO6r: fdxr_42_c + h_c + nad_c --> fdxo_42_c + nadh_c 2
7
RE2649M: h2o_m + ppcoa_m --> coa_m + h_m + ppa_m 2
8
TRDR4: gthox_c + trdrd_c --> 2.0 gthrd_c + trdox_c 2
9
SHL_c: h2s_c + h_c + ser__L_c --> cys__L_c + h2o_c 2
PLRXR: gthox_c + plrxrd_c --> 2.0 gthrd_c + plrxox_c 2
PLRXR2: plrxox_c + trdrd_c --> plrxrd_c + trdox_c 2
ASCTmr: accoa_m + succ_m --> ac_m + succoa_m 2
10
ACKrm: ac_m + atp_m --> actp_m + adp_m 1
RE2030M: accoa_m + met__L_m --> C02712_m + coa_m + h_m 1
PTArm: accoa_m + pi_m --> actp_m + coa_m 1
RE2640C: C02712_c + h

In [36]:
# ctp
add_rxn(model,'ADD_ctp','ctp_c + h2o_c --> cdp_c + h_c + pi_c')
model.objective='ADD_ctp'
ctp_flux = model.optimize().fluxes['ADD_ctp']

if ctp_flux > 1e-6:
    objective_reaction = 'ADD_ctp'
    constraint_value = 1e-6
    grate_delete_rxn = find_infinite_rxn(model,objective_reaction,constraint_value,rxn_grate)
    outputfile = 'results/quality_control/energy/'
    ctp_rxn = locating_rxn(model,check_model,objective_reaction,constraint_value,grate_delete_rxn,outputfile,unbalance_rxn)
else:
    ctp_rxn = []

rxn = model.reactions.get_by_id('ADD_ctp')
model.remove_reactions([rxn])

In [37]:
# gtp

add_rxn(model,'ADD_gtp','gtp_c + h2o_c --> gdp_c + h_c + pi_c')
model.objective='ADD_gtp'
gtp_flux = model.optimize().fluxes['ADD_gtp']

if gtp_flux > 1e-6:
    objective_reaction = 'ADD_gtp'
    constraint_value = 1e-6
    grate_delete_rxn = find_infinite_rxn(model,objective_reaction,constraint_value,rxn_grate)
    outputfile = 'results/quality_control/energy/'
    gtp_rxn = locating_rxn(model,check_model,objective_reaction,constraint_value,grate_delete_rxn,outputfile,unbalance_rxn)
else:
    gtp_rxn = []

rxn = model.reactions.get_by_id('ADD_gtp')
model.remove_reactions([rxn])


In [38]:
# utp
add_rxn(model,'ADD_utp','utp_c + h2o_c --> udp_c + h_c + pi_c')
model.objective='ADD_utp'
utp_flux = model.optimize().fluxes['ADD_utp']

if utp_flux > 1e-6:
    objective_reaction = 'ADD_utp'
    constraint_value = 1e-6
    grate_delete_rxn = find_infinite_rxn(model,objective_reaction,constraint_value,rxn_grate)
    outputfile = 'results/quality_control/energy/'
    utp_rxn = locating_rxn(model,check_model,objective_reaction,constraint_value,grate_delete_rxn,outputfile,unbalance_rxn)
else:
    utp_rxn = []

rxn = model.reactions.get_by_id('ADD_utp')
model.remove_reactions([rxn])


In [39]:
# itp
add_rxn(model,'ADD_itp','itp_c + h2o_c --> idp_c + h_c + pi_c')
model.objective='ADD_itp'
itp_flux = model.optimize().fluxes['ADD_itp']

if itp_flux > 1e-6:
    objective_reaction = 'ADD_itp'
    constraint_value = 1e-6
    grate_delete_rxn = find_infinite_rxn(model,objective_reaction,constraint_value,rxn_grate)
    outputfile = 'results/quality_control/energy/'
    itp_rxn = locating_rxn(model,check_model,objective_reaction,constraint_value,grate_delete_rxn,outputfile,unbalance_rxn)
else:
    itp_rxn = []

rxn = model.reactions.get_by_id('ADD_itp')
model.remove_reactions([rxn])

In [40]:
ppi_h2o = ppi_h2o_rule(model)

acyl_h2o = ylcoa_h2o(model)

ac_rxn = ac_rule(model)

aldehyde_ate_rxn = aldehyde_rule(model)

1
16
13
14


## 2.4 metabolites with incorrect synthetic pathways 

In [41]:
#  Determining the pathway yield of metabolites surpassed the maximum theoretical yield calculated by the ratio of the reduction degrees of substrate and product
model.reactions.get_by_id('EX_glc__D_e').bounds = (-10,1000)

over_yield_met = []
for met in model.metabolites:
    with model:
        if re.search("C[A-Z]",str(met.formula)) or re.search("C[\d]",str(met.formula)): 
            target_product = str(met.id)
            objective_reaction = 'ADD_'+target_product
            if objective_reaction in model.reactions:
                model.objective = objective_reaction
            else:
                add_rxn(model,objective_reaction,target_product + ' --> ') 
                model.objective = objective_reaction
            solution = model.optimize()
            
            if max_reduced_degree_rate(model,target_product) != 0 and not math.isnan(max_reduced_degree_rate(model,target_product)):
                if solution.fluxes[objective_reaction] != 0:
                    if (solution.fluxes[objective_reaction] - max_reduced_degree_rate(model,target_product)) > 1e-6:
                        over_yield_met.append(met.id)
                        print(target_product,solution.fluxes[objective_reaction],max_reduced_degree_rate(model,target_product))

currency_met_carbon = ['dadp', 'dutp', 'nad', 'xdp', 'imp', 'atp', 'dgtp', 'fad', 'nadp', 'idp', 'itp', 'fadh2', 'dudp', 'dgmp', 'xtp', 'dump', 'cdp', 'gtp',
                        'gdp', 'adp', 'dcdp', 'didp', 'amp', 'udp', 'dimp', 'utp', 'nadh', 'q8', 'q8h2', 'gmp', 'dtdp', 'ctp', 'ump', 'xmp', 'datp', 'dcmp',
                        'damp', 'ditp', 'dttp', 'dgdp', 'dctp', 'cmp', 'nadph', 'dtmp', 'coa', 'co2', 'fad', 'fadh2', 'q8', 'q8h2', 'mqn8', 'mql8', '2dmmql8',
                        '2dmmq8', 'q6', 'q6h2', 'fadh2', 'fad', 'mqn6', '2dmmq6', 'mql6', 'q', 'qh2', 'pqh2', 'pq', 'pqh2', 'pq', 'fadh2', 'fad', 'q10', 'q10h2',
                        'fadh2', 'fad', 'q10', 'q10h2', 'fad', 'pqh2', 'pq', 'fadh2', 'mqn7', 'mql7', 'q10', 'q10h2', 'fad', 'fad', 'pq', 'pqh2', 'pqh2', 'pqh2',
                        'pq', 'q9', 'q9h2', 'mqn11', 'mqn11', 'mqn9', 'mqn9', 'mqn10', 'mqn7', 'mqn10', 'mqn8', 'fadh2', 'pq', 'pqh2', 'pq', 'q8h2', 'q8', 'mqn4'
                        'mql4', '2dmmq4', 'mqn4', 'ficytc', 'focytc', 'focytc', 'cytcc', 'focytcc553', 'ficytcc553', 'ficytb', 'ficytC', 'focytC', 'omclo', 'omclr',
                        'omchr', 'omcho', 'ppc1o', 'ppc1r', 'focytc6', 'ficytc6', 'focytc6', 'ficytc6', 'focytC', 'ficytC', 'ficytc', 'focytc', 'ficytb5', 'focytb5',
                        'cytP450o', 'cytP450r', 'cytP450o', 'cytP450r', 'apocytc', 'cytc', 'ficytc', 'focytc', 'pqhb6l', 'hemeB3p', 'hemeB2p', 'pqb6s', 'pqhb6s',
                        'hemeX', 'hemeXp', 'pqhb6l', 'hemeB3p', 'hemeB2p', 'pqb6s', 'pqhb6s', 'focytc6', 'ficytc6', 'ficytc', 'ppcr', 'ppco', 'apocytc', 'cytc',
                        'focytcc', 'ficytcc', 'ficytb5', 'focytb5', 'HC00617', 'HC00619']
over_yield_end = []
for mi in over_yield_met:
    if mi[:-2] not in currency_met_carbon:
        over_yield_end.append(mi)

2ohph_c 0.9944751381215473 0.96


cys__L_c 17.142857142857146 13.333333333333332
met__L_c 9.075630252100822 8.0
udcpdp_c 2.25 0.7792207792207793
colipa_e 0.31088082901554426 0.2884615384615385
cys__L_e 17.14285714285715 13.333333333333332
enlipa_e 0.4197859091863151 0.41522491349480967
gthox_e 3.1858407079646014 2.926829268292683
hacolipa_e 0.2777777777777778 0.25974025974025977
halipa_e 0.3671296579575356 0.36363636363636365
met__L_e 9.075630252100844 8.0
o16a4colipa_e 0.17958097771865922 0.17341040462427743
thm_e 4.302788844621513 4.285714285714286
14glucan_c 9.642857142857142 9.6
5mtr_e 7.248322147651008 7.058823529411765
5mtr_p 7.248322147651001 7.058823529411765
5mtr_c 7.248322147651008 7.058823529411765
unaga_c 1.7088607594936702 0.7058823529411764
unagamu_c 1.4099216710182774 0.6521739130434783
pmtcoa_c 1.3366336633663423 1.3333333333333333
stcoa_c 1.2587412587412588 1.25
od2coa_c 1.2690951821386576 1.263157894736842
acolipa_p 0.3030303030303027 0.28169014084507044
acolipa_e 0.303030303030303 0.28169014084507044

In [43]:
# Determining reactions leading to incorrect metabolic pathways

end_rxn_list = []
with model:
    for target_product in over_yield_end:
        objective_reaction = 'ADD_'+target_product
        if objective_reaction in model.reactions:
            model.objective = objective_reaction
            rxn_grate[objective_reaction] = 0
        else:
            add_rxn(model,objective_reaction,target_product + ' --> ') #加入目标函数反应
            model.objective = objective_reaction
            rxn_grate[objective_reaction] = 0
        solution = model.optimize()
        constraint_value = max_reduced_degree_rate(model,target_product)

        if constraint_value > 0 and not math.isnan(constraint_value):
            if (solution.fluxes[objective_reaction] - max_reduced_degree_rate(model,target_product))>1e-6:
                
                grate_delete_rxn = find_infinite_rxn(model,objective_reaction,constraint_value,rxn_grate)
                outputfile = 'results/quality_control/pathway/'
                end_rxn = locating_rxn(model,check_model,objective_reaction,constraint_value,grate_delete_rxn,outputfile,unbalance_rxn)
                for ri in end_rxn:
                    end_rxn_list.append(ri)

1


ARTCOAL2: Rtotal2_c + coa_c --> Rtotal2coa_c 5
ARTCOAL1: Rtotal_c + coa_c --> Rtotalcoa_c 5
2
MECDPDH: 2mecdp_c + h_c --> h2mb4p_c + h2o_c 4
ARTCOAL2 1
ARTCOAL1 2
MECDPDH 3
MECDPDH: 2mecdp_c + h_c --> h2mb4p_c + h2o_c unbalance {'charge': -2.0}
1
NP1_2: nicrns_c + pi_c --> nac_c + r1p_c 5
SULRR2: h_c + nadph_c + so3_c --> h2o_c + h2s_c + nadp_c 5
2
SULRR: h_c + nadh_c + so3_c --> h2o_c + h2s_c + nad_c 5
NP1_2 1
SULRR2 2
SULRR2: h_c + nadph_c + so3_c --> h2o_c + h2s_c + nadp_c unbalance {'charge': 2.0, 'H': 2.0, 'O': -2.0}
SULRR 3
SULRR: h_c + nadh_c + so3_c --> h2o_c + h2s_c + nad_c unbalance {'charge': 2.0, 'H': 2.0, 'O': -2.0}
1
FDIPDC: frdp_c + ipdp_c --> ppi_c + udcpdp_c 5
FDIPDC 1
FDIPDC: frdp_c + ipdp_c --> ppi_c + udcpdp_c unbalance {'C': 35.0, 'H': 56.0}
1
GLUSfx: akg_c + 2.0 fdxrd_c + gln__L_c + 2.0 h_c --> 2.0 fdxox_c + 2.0 glu__L_c 5
FNOR_1: 2.0 fdxrd_c + h_c + nadp_c --> 2.0 fdxox_c + nadph_c 5
2
GALR1TRA2: gagggicolipaAR1_c + udpgal_c --> colipa_c + h_c + udp_c 4
HEPKA2: a

In [45]:
balance_rxn,chargenan_rxn,H_balance_rxn,H2O_balance_rxn,unbalance_rxn = check_rxn_balance_number(model)
cor_unri,cor_unrr,cor_balri,cor_balrr,cor_type,del_ri,del_rr,del_type = modify_delete_rxn(model,end_rxn_list,balance_rxn)

MECDPDH: 2mecdp_c + h_c --> h2mb4p_c + h2o_c MECDPDH_syn: 2mecdp_c + nadph_c --> h2mb4p_c + h2o_c + nadp_c 2
SULRR2: h_c + nadph_c + so3_c --> h2o_c + h2s_c + nadp_c SFRR: 5.0 h_c + 3.0 nadph_c + so3_c --> 3.0 h2o_c + h2s_c + 3.0 nadp_c 1
SULRR: h_c + nadh_c + so3_c --> h2o_c + h2s_c + nad_c SO3R: 5.0 h_c + 3.0 nadh_c + so3_c --> 3.0 h2o_c + h2s_c + 3.0 nad_c 1
FDIPDC: frdp_c + ipdp_c --> ppi_c + udcpdp_c UDCPDPS: frdp_c + 8.0 ipdp_c --> 8.0 ppi_c + udcpdp_c 1


HPYRR2x_1: 3.0 h_c + 2.0 hpyr_c + nadh_c --> 2.0 glyc__S_c + nad_c HPYRR2x: h_c + hpyr_c + nadh_c --> glyc__S_c + nad_c 1


In [46]:
direction_df = pd.read_excel('data/direction_check-20220428.xlsx',sheet_name='check_end',index_col='r1id')
for rxn1 in model.reactions:
    if rxn1.id in list(direction_df.index):
        rxn1.bounds = eval(direction_df.loc[rxn1.id,'r2bounds'])

In [47]:
model_dic = model_to_dict(model, sort=False)
with open('model/CSMN.json', 'w') as f:
    json.dump(model_dic,f)